In [1]:
import pandas as pd
import numpy as np

# =========================
# 1. Simple Moving Average
# =========================
def forecast_moving_average(df, n):
    df['MA_Forecast'] = df['values'].rolling(window=n).mean().shift(1)
    forecast_next = df['values'].rolling(window=n).mean().iloc[-1]
    errors = abs(df['values'][n:] - df['MA_Forecast'][n:])
    mad = errors.mean()
    return forecast_next, mad

# =========================
# 2. Weighted Moving Average
# =========================
def forecast_weighted_moving_average(df, weights):
    n = len(weights)
    reversed_weights = weights[::-1]
    sum_weights = np.sum(reversed_weights)

    last_values = df['values'].iloc[-n:].to_numpy()
    print("Data values considered (oldest to newest):", last_values)
    print("Weights (oldest to newest):", reversed_weights)

    forecast_next = np.dot(last_values, reversed_weights) / sum_weights
    print("Calculated forecast:", forecast_next)

    df['WMA_Forecast'] = df['values'].rolling(window=n).apply(
        lambda x: np.dot(x, reversed_weights) / sum_weights, raw=True
    ).shift(1)

    errors = abs(df['values'][n:] - df['WMA_Forecast'][n:])
    mad = errors.mean()
    print("MAD:", mad)
    return forecast_next, mad

# =========================
# 3. Exponential Smoothing (Simple + Textbook Holt's)
# =========================
def forecast_exponential_smoothing(df, alpha, beta=None):
    values = df['values'].to_numpy()
    n = len(values)

    if beta is None:
        # Simple Exponential Smoothing
        forecasts = [values[0]]
        for t in range(1, n):
            forecasts.append(alpha * values[t-1] + (1 - alpha) * forecasts[t-1])
        forecast_next = alpha * values[-1] + (1 - alpha) * forecasts[-1]
        errors = abs(values[1:] - np.array(forecasts[1:]))
        mad = errors.mean()
    else:
        # Holt's Linear Trend (Textbook / line-by-line replication)
        L = values[0]      # initial level
        T = 0              # initial trend
        forecasts = [L]    # forecast for period 1

        for t in range(1, n):
            # one-step-ahead forecast
            F = L + T
            forecasts.append(F)

            # update level and trend
            prev_L = L
            L = alpha * values[t] + (1 - alpha) * (L + T)
            T = beta * (L - prev_L) + (1 - beta) * T

        forecast_next = L + T   # forecast for next period

        # MAD calculated using periods 2..n (ignore first forecast)
        errors = abs(values[1:] - np.array(forecasts[1:]))
        mad = errors.mean()

    return forecast_next, mad





# =========================
# 4. Main Program
# =========================
def run_forecasting():
    # ---- Load your data ----
    file_path = r"C:\Users\300393449\OneDrive - Douglas College\Documents\4th Semester\2_Business_Statistics_II\Python\DataScratch.xlsx"
    sheet_name = "Smooth"
    
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df.columns = ['period', 'values']

    results = []

    # STEP 1 – Simple Moving Average
    ans = input("Do you want to run Simple Moving Average? (y/n): ").strip().lower()
    if ans == 'y':
        n = int(input("Enter number of periods for Moving Average: "))
        ma_forecast, ma_mad = forecast_moving_average(df.copy(), n)
        results.append(("Simple Moving Average", ma_forecast, ma_mad))

    # STEP 2 – Weighted Moving Average
    elif ans == 'n':
        ans = input("Do you want to run Weighted Moving Average? (y/n): ").strip().lower()
        if ans == 'y':
            weights_str = input("Enter weights separated by commas (e.g. 0.1,0.3,0.6): ")
            weights = [float(w) for w in weights_str.split(',')]
            wma_forecast, wma_mad = forecast_weighted_moving_average(df.copy(), weights)
            results.append(("Weighted Moving Average", wma_forecast, wma_mad))

        # STEP 3 – Simple Exponential Smoothing
        elif ans == 'n':
            ans = input("Do you want to run Simple Exponential Smoothing (alpha)? (y/n): ").strip().lower()
            if ans == 'y':
                alpha = float(input("Enter alpha (0 < alpha < 1): "))
                ses_forecast, ses_mad = forecast_exponential_smoothing(df.copy(), alpha)
                results.append(("Simple Exponential Smoothing", ses_forecast, ses_mad))

    # STEP 4 – Double Exponential Smoothing (textbook Holt’s)
    ans = input("Do you want to run Double Exponential Smoothing (alpha + beta)? (y/n): ").strip().lower()
    if ans == 'y':
        alpha = float(input("Enter alpha (0 < alpha < 1): "))
        beta = float(input("Enter beta (0 < beta < 1): "))
        des_forecast, des_mad = forecast_exponential_smoothing(df.copy(), alpha, beta)
        results.append(("Double Exponential Smoothing (Holt)", des_forecast, des_mad))

    # Display results
    if results:
        results_df = pd.DataFrame(results, columns=['Method', 'Forecast', 'MAD'])
        best_method = results_df.loc[results_df['MAD'].idxmin()]
        print("\nForecast Results:")
        print(results_df)
        print("\nBest Method:\n", best_method)
    else:
        print("No forecasting method selected.")

# =========================
# Run script
# =========================
if __name__ == "__main__":
    run_forecasting()


Do you want to run Simple Moving Average? (y/n):  n
Do you want to run Weighted Moving Average? (y/n):  n
Do you want to run Simple Exponential Smoothing (alpha)? (y/n):  n
Do you want to run Double Exponential Smoothing (alpha + beta)? (y/n):  y
Enter alpha (0 < alpha < 1):  .2
Enter beta (0 < beta < 1):  .6



Forecast Results:
                                Method  Forecast       MAD
0  Double Exponential Smoothing (Holt)   132.722  4.439688

Best Method:
 Method      Double Exponential Smoothing (Holt)
Forecast                                132.722
MAD                                    4.439688
Name: 0, dtype: object


In [17]:
    file_path = r"C:\Users\PCAdmin\OneDrive - Douglas College\Documents\4th Semester\2_Business_Statistics_II\Python\DataScratch.xlsx"
    sheet_name = "Sheet1"
    
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df.columns = ['period', 'values']

print(df)
print("Number of rows:", len(df))

   period  values
0       1     341
1       2     359
2       3     299
3       4     338
4       5     364
5       6     343
6       7     324
7       8     313
8       9     363
9      10     341
Number of rows: 10
